Импортируем модули для работы с данными

In [135]:
import requests
import lxml

from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

Базовая часть ссылки

In [136]:
base_url = "https://lifehacker.ru/topics/technology"

Далее в цикле собираем ссылки на статьи со всех 10 страниц

In [137]:
def remove_bad_url(raw_items):
    parsed_urls = []
    for item in raw_items:
        url = item.get("href")
        if "http" not in url:
            parsed_urls.append(url)
    return parsed_urls


def get_parsed_urls():
    parsed_urls = []
    for page_num in range(1, 11):
        url = f"{base_url}/?page{page_num}"
        response = requests.get(url)  # получаем контент page_num страницы
        soup = BeautifulSoup(
            response.text, "lxml"
        )  # инициализируем объект bs4 и задаем парсер lxml
        raw_items = soup.find_all("a", "lh-small-article-card__link")
        parsed_urls = remove_bad_url(raw_items)
    return parsed_urls

Для статей базовая часть ссылки изменилась

In [138]:
base_url = "https://lifehacker.ru"

In [139]:
get_parsed_urls()

['/kak-zharit-shashlyk/',
 '/top-smartfonov-v-nachale-2024/',
 '/chem-obrabotat-luk-pered-posadkoj/',
 '/muzhchiny-podvergalis-domashnemu-nasiliyu/',
 '/kak-vytashhit-kleshha/',
 '/samye-uspeshnye-filmy-2023/',
 '/pervye-otzyvy-na-furiosu/',
 '/oksid-marganca-i-zhizn-na-marse/',
 '/osnovnye-chuvstva-i-produktivnost/',
 '/prokachka-156/',
 '/atmosfera-venery-poteryala-vodu/',
 '/kak-kontrolirovat-rasxody/',
 '/riddik-furiya-s-vinom-dizelem-podtverzhden/',
 '/aksessuary-aliexpress-dlya-rabochego-stola/?erid=Kra2449mN',
 '/kak-vyvesti-lishnyuyu-zhidkost-iz-organizma/',
 '/poleznye-tovary-dlya-doma-aliexpress/?erid=Kra2449mN',
 '/xoroshie-nedavnie-rossijskie-filmy/',
 '/opros-kakimi-prilozheniyami-vy-polzuetes/',
 '/zachem-proveryat-rodinki/',
 '/10-samyx-populyarnyx-filmov-u-rossijskix-piratov-v-aprele/',
 '/oversharing/',
 '/slushaj-eto-prosto-54/?erid=LjN8KGAgK']

In [140]:
result = []
for url in tqdm(get_parsed_urls()):
    article = {}
    article_url = f"{base_url}{url}"  # подставляем часть ссылки, ведущую материал
    response = requests.get(article_url)
    soup = BeautifulSoup(response.text, "lxml")
    article["title"] = soup.find(
        "h1", class_="article-card__title"
    ).text  # Получаем текст заголовка из span в первом h1 с классом article-card__title
    text_src = [
        e.get_text()
        for e in soup.find_all(
            "div", "single-article__post-content single-article__content-container"
        )
    ]  # Получаем весь текст из div с классом single-article__post-content single-article__content-container
    article["text"] = "\n".join(text_src)
    result.append(article)

100%|██████████| 22/22 [00:07<00:00,  2.85it/s]


In [141]:
pd.set_option("display.max_colwidth", 400)


data = pd.DataFrame(result)
data.head()

,title,text
0,Как пожарить вкусный шашлык. Шеф-повар раскрывает все секреты,"Какое мясо подойдёт для шашлыкаЗалог вкусного и сочного шашлыка — правильно подобранное и замаринованное мясо.Традиционный шашлык готовится из баранины. Однако у неё специфический запах, который не всем придётся по вкусу.Куриный шашлык наверняка понравится многим, потому что получается нежным и ароматным. Для него можно взять грудку или окорочка.Для шашлыка из говядины подходит толстый край на..."
1,Названы самые продаваемые смартфоны в мире в первом квартале 2024 года,"Исследовательская компания Counterpoint Research опубликовала список самых продаваемых смартфонов в мире в первом квартале 2024 года. Все места в нём снова разделили два бренда.Самым востребованным у покупателей оказался iPhone 15 Pro Max. Второе, третье и четвёртое места заняли iPhone 15, iPhone 15 Pro и iPhone 14 соответственно.Пятым стал Samsung Galaxy S24 Ultra, что делает его самым продав..."
2,Чем обработать лук перед посадкой,"Зачем обрабатывать лук перед посадкойПравильная подготовка лука-севка может повысить урожай. Как правило, обработанные луковки лучше противостоят заболеваниям, например ложной мучнистой росе и фузариозной гнили. А ещё севок, замоченный в дезинфицирующих растворах, становится «невкусным» для некоторых вредителей вроде луковых корневых клещей, трипсов и луковой мухи.Как и чем обработать лук пере..."
3,"«В меня летал стул»: мужчины рассказали, как подвергались домашнему насилию и абьюзу","На волне популярности мини-сериал «Оленёнок», в центре сюжета которого сталкинг и одержимость другим человеком, в Сети запустили откровенный тред о домашнем насилии и абьюзе, причём со стороны женщин. В комментариях многие мужчины рассказали о своих случаях, когда нездоровые отношения выливались в побои, угрозы и постоянное унижение.Это довольно тяжёлый тред. В нём немало жутких деталей. Мы вы..."
4,Как вытащить клеща правильно,"Почему лучше обратиться за помощьюЕсли есть возможность, идите к врачу, как только увидите присосавшегося клеща на теле. Доктор знает, как правильно вытащить паразита и обработать кожу. Татьяна Лошкарева Заслуженный врач РФ, врач-инфекционист высшей квалификационной категории «Мединцентра». Оказать помощь вам могут в ближайшем травмпункте. Отправляйтесь туда незамедлительно. Если не знаете..."
